In [1]:
# Hidden TimeStamp
import time, datetime
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
print('Last Run: {}'.format(st))

Last Run: 2016-03-12 21:13:11


# Testing Code

A guide for testing code prior to submitting pull request.

## Testing code with `nose`

The current testing utility is `nose`.  From the root directory, you can test all files prepended with "test_" by running:

    $ nosetests
   
There are three types of tests contained in the source `lamana` directory:

1. module tests: normal test files located in the "./tests" directory
1. model tests: test files specific for custom models, located in "./models/tests"
1. controls: .csv files located "./tests/controls_LT"

Models tests are separated to support an extensibile design for author contributions. This design enables authors to create models and tests together with a single pull request to the standard module directory.

Tests for the `utils` module writes and removes temporary files in a root directory called "export".  If this directory does not exist, one will be created.  These test check that writing and reading of files are consistent.  Temporary files are prefixed with "temp", but should be removed by these test functions.

### Control files


LamAna maintains .csv files with expected data for different lamanate configurations.  These files are tested with the `test_controls` module.  This module reads each control file and parses information such as layer numbers, number of points per layer and geometry.  Control files are named by these variables.

Controls files can be created manually, but it may be simpler to make and then edit a starter file.  This process can be expedited for multiple files by passing LaminateModels into the `utils.tools.write_csv()` function.  This function will create a csv file for every LaminateModel, which can be altered as desired and tested by copying into the "lamana/tests/controls_LT" folder.

## Coverage

We use the following tools and commands to assess test coverage.  `nose-cov` helps to combine coverage reports for sub-packages automatically.  The remaining flags will report missing lines for the source directory. 

```
$ pip install coverage, nose-cov
$ nosetests --with-cov --cov lamana
```

or

```    
$ nosetests --with-cov --cov-report term-missing --cov lamana
```
LamAna aims for the highest "reasonable" coverage for core modules.  A separate ideology must be developed for testing `output_` as plots are tricky to test fully. 

## Pre-Deployment

Prior to a release, it is fitting to test any API demonstration notebooks in a  development virtual environment and release branch.  A simple way to validate notebook cells without errors is to use the [`runipy` tool](https://github.com/paulgb/runipy).  This tool will run notebook cells in the command prompt and halt if errors are found.  The command is simple to apply to a notebook:

```
$ pip install runipy
$ runipy <demo>.ipynb
```

If no errors were found, your tested API works, and you can advance in the release workflow.